In [1]:
import pandas as pd

In [4]:
df = pd.read_csv(
    "../SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)


In [5]:
sizeData = len(df.index)
trainSize = int(0.5*sizeData)
trainDF = df.sample(trainSize)
trainDF

,spam,text
2049,ham,So dont use hook up any how
1905,ham,Wah... Okie okie... Muz make use of e unlimite...
4584,spam,For ur chance to win a £250 wkly shopping spre...
2784,ham,"Just arrived, see you in a couple days &lt;3"
3344,ham,"I haven't forgotten you, i might have a couple..."
...,...,...
4008,ham,I'm reaching home in 5 min.
1482,ham,"I'm a guy, browsin is compulsory"
993,ham,Up to ü... Ü wan come then come lor... But i d...
4225,ham,Double eviction this week - Spiral and Michael...


In [7]:
trainDFSpam = trainDF[trainDF["spam"]=="spam"]
trainDFSpam["text"]=trainDFSpam["text"].str.lower()
dfSplitSpam = trainDFSpam.text.str.split(expand=True).stack().value_counts().to_frame()

trainDFHam = trainDF[trainDF["spam"]=="ham"]
trainDFHam["text"]=trainDFHam["text"].str.lower()
dfSplitHam = trainDFHam.text.str.split(expand=True).stack().value_counts().to_frame()

total = dfSplitSpam.join(dfSplitHam,lsuffix='_spam', rsuffix='_ham')
total.fillna(0.001,inplace=True)
spamTotal=total["count_spam"].sum()
hamTotal = total["count_ham"].sum()
total["frequence"]=(total["count_spam"]/spamTotal)/(total["count_ham"]/hamTotal)

      spam                                               text
4584  spam  for ur chance to win a £250 wkly shopping spre...
598   spam  you have an important customer service announc...
1050  spam  18 days to euro2004 kickoff! u will be kept in...
4376  spam  ur tonexs subscription has been renewed and yo...
1154  spam  1000's of girls many local 2 u who r virgins 2...
      count
to      350
a       199
call    166
you     132
your    125
     spam                                               text
2049  ham                        so dont use hook up any how
1905  ham  wah... okie okie... muz make use of e unlimite...
2784  ham       just arrived, see you in a couple days &lt;3
3344  ham  i haven't forgotten you, i might have a couple...
3434  ham  christmas is an occasion that is celebrated as...
     count
i     1092
you    840
to     788
a      539
the    533
      count_spam  count_ham
to           350      788.0
a            199      539.0
call         166      113.0
you         

C:\Users\melos\AppData\Local\Temp\ipykernel_22344\245926628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFSpam["text"]=trainDFSpam["text"].str.lower()
C:\Users\melos\AppData\Local\Temp\ipykernel_22344\245926628.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFHam["text"]=trainDFHam["text"].str.lower()


In [15]:
import pandas as pd
import numpy as np

# Étape 1 : Préparer les données
# Imaginons que tu aies déjà ton DataFrame 'trainDF' avec les colonnes 'text' et 'spam' où 'spam' = 1 pour les spams et 'ham' = 0 pour les non-spams.

# Filtrage des messages spam et non-spam
trainDFSpam = trainDF[trainDF["spam"] == 1]
trainDFHam = trainDF[trainDF["spam"] == 0]

# Convertir les textes en minuscules
trainDFSpam["text"] = trainDFSpam["text"].str.lower()
trainDFHam["text"] = trainDFHam["text"].str.lower()

# Compter les occurrences des mots
dfSplitSpam = trainDFSpam.text.str.split(expand=True).stack().value_counts().to_frame()
dfSplitHam = trainDFHam.text.str.split(expand=True).stack().value_counts().to_frame()

# Fusionner les comptages des mots
total = dfSplitSpam.join(dfSplitHam, lsuffix='_spam', rsuffix='_ham').fillna(0.001)

# Calculer la fréquence de chaque mot dans chaque classe
spamTotal = total["count_spam"].sum()
hamTotal = total["count_ham"].sum()

total["p_spam"] = total["count_spam"] / spamTotal
total["p_ham"] = total["count_ham"] / hamTotal

# Étape 2 : Calculer les probabilités a priori (P(spam) et P(ham))
total_documents = len(trainDF)  # Nombre total de documents
p_spam = len(trainDFSpam) / total_documents  # Probabilité a priori d'un document spam
p_ham = len(trainDFHam) / total_documents  # Probabilité a priori d'un document ham

# Étape 3 : Calculer les probabilités conditionnelles pour chaque mot
total["p_spam"] = total["count_spam"] / spamTotal
total["p_ham"] = total["count_ham"] / hamTotal

# Étape 4 : Fonction pour calculer les probabilités d'un document
def calculate_probabilities(document, p_spam, p_ham, total):
    p_document_spam = np.log(p_spam)  # Log de P(spam)
    p_document_ham = np.log(p_ham)  # Log de P(ham)

    # Calcul des probabilités conditionnelles pour chaque mot du document
    for word in document.split():
        if word in total.index:
            p_document_spam += np.log(total.loc[word, 'p_spam'])
            p_document_ham += np.log(total.loc[word, 'p_ham'])
        else:
            p_document_spam += np.log(0.001)  # Lissage pour les mots inconnus
            p_document_ham += np.log(0.001)
    
    return p_document_spam, p_document_ham

# Étape 5 : Fonction pour faire la prédiction
def predict(document, p_spam, p_ham, total):
    p_spam_doc, p_ham_doc = calculate_probabilities(document, p_spam, p_ham, total)
    if p_spam_doc > p_ham_doc:
        return "spam"
    else:
        return "ham"

# Exemple d'utilisation : prédire pour un nouveau document
test_document = "for ur chance to win a £250 wkly shopping"
prediction = predict(test_document, p_spam, p_ham, total)
print(f"Le document '{test_document}' est classé comme : {prediction}")

# Étape 6 : Évaluation du modèle
# Prédire pour tous les documents de test et comparer avec les vraies étiquettes
correct_predictions = 0
for _, row in df.iterrows():
    predicted_label = predict(row['text'], p_spam, p_ham, total)
    if predicted_label == row['spam']:
        correct_predictions += 1

accuracy = correct_predictions / len(df)
print(f"Précision du modèle : {accuracy * 100:.2f}%")


C:\Users\melos\AppData\Local\Temp\ipykernel_22344\554863744.py:40: RuntimeWarning: divide by zero encountered in log
  p_document_spam = np.log(p_spam)  # Log de P(spam)
C:\Users\melos\AppData\Local\Temp\ipykernel_22344\554863744.py:41: RuntimeWarning: divide by zero encountered in log
  p_document_ham = np.log(p_ham)  # Log de P(ham)


Le document 'for ur chance to win a £250 wkly shopping' est classé comme : ham
Précision du modèle : 86.59%
